# Pandas 2.0 using Pyarrow backend

In [1]:
!pip install pandas==2.2.3
!pip install pyarrow==19.0.0

In [2]:
import pandas as pd
import time
import statistics

In [3]:
%run utilities.ipynb

In [4]:
config_engine = "Pyarrow" # Pyarrow or C

if config_engine == "Pyarrow":
    kwargs_read_csv = {"engine":"pyarrow"}
    filename = "Pandas2.0Pyarrow"
else:
    kwargs_read_csv = {}
    filename = "Pandas2.0Numpy"

filename += ".csv"

## Reading file

In [5]:
total_read = []
for i in range(REPETITIONS): 
    start_time = time.time()
    df = pd.read_csv("Imdb Movie Dataset.csv", **kwargs_read_csv)
    time_reading_file = time.time() - start_time
    total_read.append(time_reading_file)
print("--- %s seconds ---" % sum(total_read))

--- 138.25511598587036 seconds ---


In [6]:
df

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,...,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,7/15/2010,825532764,148,False,160000000,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,11/5/2014,701729206,169,False,165000000,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,7/16/2008,1004558444,152,False,185000000,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,12/15/2009,2923706026,162,False,237000000,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,4/25/2012,1518815515,143,False,220000000,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,905156,鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头,0.000,0,Released,None,0,0,False,0,...,zh,鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头,None,0.600,None,None,None,None,None,None
1048571,905157,MILF & Cookies 3,0.000,0,Released,None,0,0,True,0,...,en,MILF & Cookies 3,None,0.600,None,None,None,None,None,None
1048572,905158,The Choice of Staying,0.000,0,Released,10/8/2020,0,0,False,0,...,it,The Choice of Staying,None,0.600,None,Documentary,None,None,"English, Italian, Swedish",None
1048573,905161,Luisa Schluckt Schon Wieder,0.000,0,Released,5/13/2016,0,0,True,0,...,de,Luisa Schluckt Schon Wieder,None,0.600,None,None,John Thompson Productions,None,None,None


In [7]:
df.memory_usage(index=True, deep = True).sum()

np.int64(1010114540)

## Cleaning data

Things that we are going to remove and measure only once
- No release Date and status is released
- Runtime 0 or Null
- Budget 0. It has to cost something
- It has to have a genre

In [8]:
data_cleaned = df[~((df["status"] == "Released") & (df["release_date"].isnull()))]
data_cleaned = data_cleaned[(data_cleaned["runtime"] > 0) & (data_cleaned["runtime"].notnull())]
data_cleaned = data_cleaned[(data_cleaned["budget"] > 0)]
data_cleaned = data_cleaned[(df["genres"].notnull())]

/tmp/ipykernel_35958/1295375744.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_cleaned = data_cleaned[(df["genres"].notnull())]


In [9]:
data_cleaned

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,...,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,7/15/2010,825532764,148,False,160000000,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,11/5/2014,701729206,169,False,165000000,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,7/16/2008,1004558444,152,False,185000000,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,12/15/2009,2923706026,162,False,237000000,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,4/25/2012,1518815515,143,False,220000000,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048326,904802,Last Ride of The Day,0.000,0,Released,11/14/2020,0,17,False,3000,...,id,Boncengan,Nadia (15) spends her last day riding a motorb...,0.600,None,"Drama, Family","Halaman Belakang Film, Sineman Films, UMN (Uni...",Indonesia,Indonesian,None
1048549,905056,Saint Nicolas chez les Baltus,0.000,0,Released,12/1/1993,32896,4,False,32896,...,en,Saint Nicolas chez les Baltus,the Animated Short Film by Stéphane Aubier and...,0.600,None,Animation,None,France,French,None
1048550,905057,Les Baltus Au Cirque,0.000,0,Released,8/20/1999,27378,6,False,27378,...,en,Les Baltus Au Cirque,the Animated Short Film by Stéphane Aubier and...,0.600,None,Animation,None,France,French,None
1048555,905063,Science Please! : The Force of Water,0.000,0,Released,1/1/1998,43134,1,False,43134,...,en,Science Please! : The Force of Water,"A clip in the Science Please! collection, The ...",0.600,None,Animation,ONF | NFB,Canada,English,water


## Select

In [10]:
total_select = []
for i in range(REPETITIONS): 
    start_time = time.time()
    data_cleaned = data_cleaned[[c for c in data_cleaned.columns if c not in [
    "overview", "tagline","genres", "production_companies","production_countries","spoken_languages", "keywords"]
                                   ]]
    time_select = time.time() - start_time
    total_select.append(time_select)
print("--- %s seconds ---" % sum(total_select))

--- 0.1979837417602539 seconds ---


## Filter

In [11]:
total_filter = []
for i in range(REPETITIONS):
    start_time = time.time()
    df_filter = data_cleaned[data_cleaned["budget"] > 550000000]
    time_filter = time.time() - start_time
    total_filter.append(time_filter)
print("--- %s seconds ---" % sum(total_filter))

--- 0.015083074569702148 seconds ---


In [12]:
df_filter

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,imdb_id,original_language,original_title,popularity
507929,1057999,Enea,0.0,0,Released,9/21/2023,0,115,False,888000000,tt27219440,it,Enea,4.319
707555,1224207,Adventures in Bora Bora,0.0,0,Released,8/23/2023,3000000000,5,False,800000000,None,en,Adventures in Bora Bora,0.000


## Aggregation

In [13]:
total_agg = []
for i in range(REPETITIONS):
    start_time = time.time()
    df_agg = data_cleaned[["status","runtime"]].groupby(["status"]).mean()
    time_agg = time.time() - start_time
    total_agg.append(time_agg)
print("--- %s seconds ---" % sum(total_agg))

--- 0.17758917808532715 seconds ---


In [14]:
df_agg

,runtime
status,
Canceled,27.285714
In Production,38.623446
Planned,41.858407
Post Production,41.353968
Released,71.266654
Rumored,63.000000


## Sorting

In [15]:
total_sort = []
for i in range(REPETITIONS):
    start_time = time.time()
    df_sort = data_cleaned.sort_values(["revenue"])
    time_sort = time.time() - start_time
    total_sort.append(time_sort)
print("--- %s seconds ---" % sum(total_sort))

--- 0.34035730361938477 seconds ---


In [16]:
df_sort

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,imdb_id,original_language,original_title,popularity
207944,64767,Deadrise,1.500,2,Released,4/5/2011,0,95,False,1000000,tt1552192,en,Deadrise,0.840
184774,763484,Die by the Sword,5.500,2,Released,10/6/2020,0,90,False,500000,tt12484058,en,Die by the Sword,1.216
518782,1400372,Without Mercy,0.000,0,Post Production,None,0,20,False,50000,tt30477702,en,Without Mercy,0.600
184738,1036417,Sunflower's Friend,7.500,2,Released,9/25/2022,0,2,False,1000,tt21236962,en,Sunflower's Friend,0.701
184636,323867,Spy,5.500,2,Released,8/9/2011,0,110,False,5000000,tt0372538,en,Spy,1.497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,76600,Avatar: The Way of Water,7.654,9830,Released,12/14/2022,2320250281,192,False,460000000,tt1630029,en,Avatar: The Way of Water,241.285
15,299534,Avengers: Endgame,8.263,23857,Released,4/24/2019,2800000000,181,False,356000000,tt4154796,en,Avengers: Endgame,91.756
3,19995,Avatar,7.573,29815,Released,12/15/2009,2923706026,162,False,237000000,tt0499549,en,Avatar,79.932
707555,1224207,Adventures in Bora Bora,0.000,0,Released,8/23/2023,3000000000,5,False,800000000,None,en,Adventures in Bora Bora,0.000


## Complex Execution

In [17]:
total_complex = []
for i in range(REPETITIONS):
    start_time = time.time()
    data_cleaned["release_date"] = pd.to_datetime(data_cleaned["release_date"], format="mixed")
    revenue_2024 = data_cleaned[(data_cleaned["release_date"] >= '2024-01-01') & (data_cleaned["release_date"] <= '2024-12-31')] 
    revenue_2024_non_en = revenue_2024[revenue_2024["original_language"] != "en"]
    max_revenue_2024_by_language = revenue_2024[["original_language","revenue"]].groupby(["original_language"]).max()
    max_revenue_2024_by_language_non = revenue_2024_non_en[["original_language","revenue"]].groupby(["original_language"]).max()
    time_complex = time.time() - start_time
    total_complex.append(time_complex)
print("--- %s seconds ---" % sum(total_complex))
max_revenue_2024_by_language_non

--- 0.6554312705993652 seconds ---


,revenue
original_language,
ar,10
as,0
az,0
bg,0
bn,0
ca,0
cs,0
da,0
de,0


## Saving metrics

In [18]:
data_list = ["sum","avg","min","max"]
write_list = []
for x in data_list:
    data = {}
    fun = sum if x == "sum" else statistics.mean if x == "avg" else min if x == "min" else max
    data["type"] = x
    data["read_file"] = fun(total_read)
    data["memory_size_read"] = df.memory_usage(index=True, deep = True).sum()
    data["select"] = fun(total_select)
    data["memory_size_select"] = data_cleaned.memory_usage(index=True, deep = True).sum()
    data["filter"] = fun(total_filter)
    data["memory_size_filter"] = df_filter.memory_usage(index=True, deep = True).sum()
    data["agg"] = fun(total_agg)
    data["memory_size_agg"] = df_agg.memory_usage(index=True, deep = True).sum()
    data["sort"] = fun(total_sort)
    data["memory_size_sort"] = df_sort.memory_usage(index=True, deep = True).sum()
    data["complex"] = fun(total_complex)
    data["memory_size_complex"] = max_revenue_2024_by_language_non.memory_usage(index=True, deep = True).sum()
    data["repetitions"] = REPETITIONS
    write_list.append(data)
write_csv(filename,write_list)